# Utility scripts for WEAV Study - Walking Energy Audit from Video

Use these functions to process & analyze walking biomechanics data


Ricky Pimentel & Roshan Ragu

Applied Biomechanics Laboratory

## Import Libraries & Connect to Gdrive

In [1]:
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import scipy.io as sio
import scipy.stats as stats
from datetime import datetime, timedelta

sty = 'seaborn'
mpl.style.use(sty)

C:\Users\rosha\AppData\Local\Temp\ipykernel_16532\1537878433.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use(sty)


In [2]:
""" 
Data avaible at 
https://drive.google.com/drive/folders/1r-qjnhacvAieNrcHOblaoXUf6lKDaV6G?usp=sharing
"""

' \nData avaible at \nhttps://drive.google.com/drive/folders/1r-qjnhacvAieNrcHOblaoXUf6lKDaV6G?usp=sharing\n'

Metabolic Analysis

In [3]:
# def CalcWatts(VO2, VCO2):
#   # calculate watts from volume of O2 consumed and CO2 produced
#   # based on the Brockway equation - http://www.ncbi.nlm.nih.gov/pubmed/3429265

#   W = (VO2 /1000 * 16.5 + VCO2 /1000 * 4.51) * 1000 / 60 

#   return W

def GetRMR(fname):
  # load and transform resting metabolic rate (RMR) data
  # these metabolics files differ from the active trials by length
  # RMR's should be ~ 5 min, while active trials contain multiple 5-min trials

  RMR = pd.read_csv(fname) # load data as DF
    
  # pull VO2 and time data
  VO2 = RMR.loc[2:len(RMR),'VO2'].values
  VCO2 = RMR.loc[2:len(RMR),'VCO2'].values
  t = RMR.loc[2:len(RMR),'t'].values.tolist()
  W = (VO2 /1000 * 16.5 + VCO2 /1000 * 4.51) * 1000 / 60 # calculate watts
  
  # find rows after 3 min, append to T
  T = []
  time = []
  for i, x in enumerate(t):
    if x is np.nan: 
      continue # skip NaNs
    # print(timedelta(hours=t[i].hour, minutes=t[i].minute, seconds=t[i].second).total_seconds()) # print seconds elapsed
    time.append(timedelta(hours=t[i].hour, minutes=t[i].minute, seconds=t[i].second).total_seconds()) # save seconds elapsed
    if t[i].minute >= 3:
      T.append(i)
            
  # calculate average RMR and make array
  AvgVO2 = np.mean(VO2[T])
  AvgVCO2 = np.mean(VCO2[T])
  AvgW = np.mean(W[T])
  Seq = list(range(0, len(T)))
  A = np.ones((len(T)))
  AvgRMR_array = A * AvgW
            
  RMRdata = {} # save data into dict
  RMRdata['AvgVO2'] = AvgVO2
  RMRdata['AvgVCO2'] = AvgVCO2
  RMRdata['AvgW'] = AvgW
  RMRdata['VO2'] = VO2
  RMRdata['VCO2'] = VCO2
  RMRdata['W'] = W
  RMRdata['Time'] = time
  RMRdata['Raw'] = RMR

  return RMRdata



def GetMet(fname):
  # load and transform resting metabolic rate (RMR) data
  # these metabolics files differ from the active trials by length
  # RMR's should be ~ 5 min, while active trials contain multiple 5-min trials

  Active = pd.read_csv(fname)

  # pull VO2 and time data
  VO2 = Active.loc[2:len(Active),'VO2'].values
  VCO2 = Active.loc[2:len(Active),'VO2'].values
  t = Active.loc[2:len(Active),'t'].values.tolist()
  W = (np.multiply(np.divide(VO2, 1000), 16.58) + 
                np.multiply(np.divide(VCO2, 1000), 4.51)) * 1000 / 60  # calculate watts
  Mkr = Active.loc[2:len(Active),'Marker'].values.tolist()
    
  # convert start and end times to datetimes
  today = dt.datetime.today()
  ProtocolStartTime = dt.datetime.combine(today, t[0])
  ProtocolEndTime = dt.datetime.combine(today, t[-1])

  plotT = 1
  if plotT == 1:
    plt.figure(figsize=(10,10))

  Trials = ['Flat_08','Flat_10','Flat_12','Up_10','Down_10']
  TrialData = {}
  for trial in Trials: 
    
    today = dt.datetime.today()
      
    # add start and end times
    StartStr = 'Start_' + trial
    EndStr = 'End_' + trial
        
    # pull start and end times of trial
    StartInd = Mkr.index(StartStr)
    EndInd = Mkr.index(EndStr)

    # get measures during trial
    TrialVO2 = VO2[StartInd:EndInd]
    TrialVCO2 = VCO2[StartInd:EndInd]
    TrialW = W[StartInd:EndInd]
    
    # create start and end times as datetimes
    StartTime = dt.datetime.combine(today, t[StartInd])
    # EndTime = dt.datetime.combine(today, t[EndInd])
    #  TrialTime = EndTime-StartTime

    # convert to seconds
    TrlSec = []
    for i in t[StartInd:EndInd]:
        TrlTime = dt.datetime.combine(today, i)
        ts = TrlTime - StartTime
        TrlSec.append(ts.total_seconds())
    
    # find final 2 min of trial
    Final2Min = [x for x in TrlSec if x >= 180]
    Final2MinInd = TrlSec.index(Final2Min[0])
    
    # average VO2 over final 2 min 
    TrialVO2Avg = np.mean(TrialVO2[Final2MinInd:EndInd])
    TrialVCO2Avg = np.mean(TrialVCO2[Final2MinInd:EndInd])
    TrialWAvg = np.mean(TrialW[Final2MinInd:EndInd])
    
    tdata = {}
    tdata['AvgVO2'] = TrialVO2Avg
    tdata['AvgVCO2'] = TrialVCO2Avg
    tdata['AvgW'] = TrialWAvg
    tdata['VO2'] = TrialVO2
    tdata['VCO2'] = TrialVCO2
    tdata['W'] = TrialW
    tdata['Time'] = TrlSec
    tdata['Raw'] = Active

    TrialData[trial] = tdata # create sub dict within greater dict

    if plotT == 1:
      plt.plot(tdata['Time'], tdata['W'], label=trial)
      final2min = np.arange(180, 300)
      plt.plot(final2min, np.ones(len(final2min))*tdata['AvgW'], lw=5)

    
  if plotT == 1:
    plt.legend()
    plt.xlabel('time (s)')
    plt.ylabel('Watts')
    plt.show()

  return TrialData

  # Rest Time Analysis

#     # extract & analyze rest times   
#     StartInds = []
#     i = 0
#     for v in Mkr:
#         if 'start' in str(v):
#             StartInds.append(i)
#         i = i + 1
        
#     EndInds = []
#     i = 0
#     for v in Mkr:
#         if 'end' in str(v):
#             EndInds.append(i)
#         i = i + 1
        
#     del(StartInds[0])
#     del(EndInds[-1])
#     RestTime = []
#     for i in range(len(StartInds)):
#         starts = dt.datetime.combine(today, t[StartInds[i]])
#         ends = dt.datetime.combine(today, t[EndInds[i]])
#         NumSec = starts - ends
#         RestTime.append(NumSec.seconds)

#     Subjects[s]['RestTime'] = RestTime
    


In [4]:
#%% Rest Time Analysis
# RestTimes = []
# for s in Subjects:
#     for x in Subjects[s]['RestTime']:
#         RestTimes.append(x) 
        
# AvgRestTimes = np.mean(RestTimes)
# SDRestTimes = np.std(RestTimes)
# MinRestTimes = np.min(RestTimes)
# MaxRestTimes = np.max(RestTimes)

# print('Rest Times')
# print('Avg = ' + str(AvgRestTimes))
# print('SD = ' + str(SDRestTimes))
# print('Min = ' + str(MinRestTimes))
# print('Max = ' + str(MaxRestTimes))